In [1]:
import random

import numpy
import pandas as pd
import numpy as np
import numba as nb
from modules.engine.mamdani.driver import MamdaniInferenceEnginer as mmdn;
from modules.fis import FIS
from modules.membershipFunctions.triangle import Triangle
import sklearn
from numpy import unique
from sklearn.cluster import MeanShift
from sklearn.cluster import KMeans
from numpy import where
from numba import jit, cuda, errors,njit,prange,typeof
from timeit import default_timer as timer


In [2]:
data=pd.read_csv('./data/wine.data',sep=',',names=[
    'type','x1','x2','x3','x4','x5','x6','x7','x8','x9','x10','x11','x12','x13'
    ])

# start normalize
copy_data = data.copy()
for i in range(1,14):
    column = 'x'+str(i)
    copy_data[column] = copy_data[column] / copy_data[column].abs().max()
#end normalize


#start select data for test
percentage = 0.05
test_index = []
train_index =[]
while len(test_index) < np.floor(data.shape[0]*percentage):
    rand = random.randrange(0,data.shape[0])
    if rand not in test_index:
        test_index.append(rand)
for i in range(0,data.shape[0]):
    if i not in test_index:
        train_index.append(i)
test_index = np.array(test_index)
train_index = np.array(train_index)

#end select data for test

In [3]:
train_index.shape

(170,)

In [4]:
all_roles = []
num_all_roles =1
for i in range(1,14):
    column = 'x'+str(i)
    X = []
    for data_index in train_index:
        X.append([
            copy_data[column][data_index],
        ])
    X = np.array(X)
    model = MeanShift()
    yhat = model.fit_predict(X)
    clusters = unique(yhat)
    dimention_triagle = []
    for j,fazzy_index in enumerate(clusters):
        row_ix = where(yhat == fazzy_index)
        # all_roles[i-1][j][0] = np.min(X[row_ix,0][0])
        # all_roles[i-1][j][1] = np.mean(X[row_ix,0][0])
        # all_roles[i-1][j][2] = np.max(X[row_ix,0][0])
        dimention_triagle.append([np.min(X[row_ix,0][0]),np.mean(X[row_ix,0][0]),np.max(X[row_ix,0][0])])
    all_roles.append(dimention_triagle)
    num_all_roles = num_all_roles * clusters.shape[0]
print(num_all_roles)
print(all_roles)

1259712
[[[0.8799730276466622, 0.9213893827009133, 1.0], [0.743762643290627, 0.828955808101574, 0.8786244099797706]], [[0.12758620689655173, 0.27298292601272184, 0.3724137931034483], [0.5586206896551724, 0.703784693019344, 1.0], [0.39827586206896554, 0.46684350132626, 0.5465517241379311]], [[0.5820433436532507, 0.6941055059794815, 0.758513931888545], [0.7616099071207431, 0.811460355774781, 0.8885448916408669], [0.9040247678018576, 0.9669762641898866, 1.0], [0.4210526315789474, 0.5180598555211559, 0.563467492260062]], [[0.5399999999999999, 0.6829342723004694, 1.0], [0.35333333333333333, 0.48964285714285716, 0.5366666666666667]], [[0.43209876543209874, 0.591091091091091, 0.7098765432098766], [0.7160493827160493, 0.7577895355673133, 0.8580246913580247], [0.9320987654320988, 0.9320987654320988, 0.9320987654320988]], [[0.6314432989690723, 0.7384857410630608, 1.0], [0.25257731958762886, 0.40637300843486407, 0.4948453608247423], [0.49742268041237114, 0.5573792729245796, 0.6237113402061856]], 

In [5]:
roles_label = np.zeros((
    np.array(all_roles[0]).shape[0],
    np.array(all_roles[1]).shape[0],
    np.array(all_roles[2]).shape[0],
    np.array(all_roles[3]).shape[0],
    np.array(all_roles[4]).shape[0],
    np.array(all_roles[5]).shape[0],
    np.array(all_roles[6]).shape[0],
    np.array(all_roles[7]).shape[0],
    np.array(all_roles[8]).shape[0],
    np.array(all_roles[9]).shape[0],
    np.array(all_roles[10]).shape[0],
    np.array(all_roles[11]).shape[0],
    np.array(all_roles[12]).shape[0],
))
roles_data = np.zeros((
    np.array(all_roles[0]).shape[0],
    np.array(all_roles[1]).shape[0],
    np.array(all_roles[2]).shape[0],
    np.array(all_roles[3]).shape[0],
    np.array(all_roles[4]).shape[0],
    np.array(all_roles[5]).shape[0],
    np.array(all_roles[6]).shape[0],
    np.array(all_roles[7]).shape[0],
    np.array(all_roles[8]).shape[0],
    np.array(all_roles[9]).shape[0],
    np.array(all_roles[10]).shape[0],
    np.array(all_roles[11]).shape[0],
    np.array(all_roles[12]).shape[0],
))
a=typeof(1+2)

In [6]:
@njit()
def triangle(x,a,b,c):
    if a>x:
        return 0.0
    elif a<=x and x<= b:
        return np.divide((np.subtract(x,a)),(np.subtract(b,a)))
    elif b<=x and x<= c:
        return np.divide(np.subtract(c,x),np.subtract(c,b))
    elif c<=x:
        return 0.0
    return 0.0

In [10]:
@njit()
def test():
    print(all_roles[0][0])
test()

TypingError: Failed in nopython mode pipeline (step: ensure features that are in use are in a valid form)
[1mThe use of a reflected list(reflected list(reflected list(float64)<iv=None>)<iv=None>)<iv=None> type, assigned to variable 'all_roles' in globals, is not supported as globals are considered compile-time constants and there is no known way to compile a reflected list(reflected list(reflected list(float64)<iv=None>)<iv=None>)<iv=None> type as a constant.
[1m
File "C:\Users\Home\AppData\Local\Temp\ipykernel_17936\190289365.py", line 3:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [7]:
def logistic_regression(n,a_x1,a_x2,a_x3,a_x4,a_x5,a_x6,a_x7,a_x8,a_x9,a_x10,a_x11,a_x12,a_x13,a_type):
    for i in prange(n):
        data_index = train_index[i]
        max = np.float64(0)
        for x1 in range(np.array(all_roles[0]).shape[0]):
            for x2 in range(np.array(all_roles[1]).shape[0]):
                for x3 in range(np.array(all_roles[2]).shape[0]):
                    for x4 in range(np.array(all_roles[3]).shape[0]):
                        for x5 in range(np.array(all_roles[4]).shape[0]):
                            for x6 in range(np.array(all_roles[5]).shape[0]):
                                for x7 in range(np.array(all_roles[6]).shape[0]):
                                    for x8 in range(np.array(all_roles[7]).shape[0]):
                                        for x9 in range(np.array(all_roles[8]).shape[0]):
                                            for x10 in range(np.array(all_roles[9]).shape[0]):
                                                for x11 in range(np.array(all_roles[10]).shape[0]):
                                                    for x12 in range(np.array(all_roles[11]).shape[0]):
                                                        for x13 in range(np.array(all_roles[12]).shape[0]):
                                                            min = np.float64(1)
                                                            min = min * triangle(a_x1[data_index], *all_roles[0][x1])
                                                            min = min * triangle(a_x2[data_index], *all_roles[1][x2])
                                                            min = min * triangle(a_x3[data_index], *all_roles[2][x3])
                                                            min = min * triangle(a_x4[data_index], *all_roles[3][x4])
                                                            min = min * triangle(a_x5[data_index], *all_roles[4][x5])
                                                            min = min * triangle(a_x6[data_index], *all_roles[5][x6])
                                                            min = min * triangle(a_x7[data_index], *all_roles[6][x7])
                                                            min = min * triangle(a_x8[data_index], *all_roles[7][x8])
                                                            min = min * triangle(a_x9[data_index], *all_roles[8][x9])
                                                            min = min * triangle(a_x10[data_index], *all_roles[9][x10])
                                                            min = min * triangle(a_x11[data_index], *all_roles[10][x11])
                                                            min = min * triangle(a_x12[data_index], *all_roles[11][x12])
                                                            min = min * triangle(a_x13[data_index], *all_roles[12][x13])
                                                            if(roles_data[x1][x2][x3][x4][x5][x6][x7][x8][x9][x10][x11][x12][x13]<min):
                                                                roles_label[x1][x2][x3][x4][x5][x6][x7][x8][x9][x10][x11][x12][x13] = a_type[data_index]
                                                                roles_data[x1][x2][x3][x4][x5][x6][x7][x8][x9][x10][x11][x12][x13] = min

In [9]:
a_x1 = np.array(copy_data["x1"])
a_x2 = np.array(copy_data["x2"])
a_x3 = np.array(copy_data["x3"])
a_x4 = np.array(copy_data["x4"])
a_x5 = np.array(copy_data["x5"])
a_x6 = np.array(copy_data["x6"])
a_x7 = np.array(copy_data["x7"])
a_x8 = np.array(copy_data["x8"])
a_x9 = np.array(copy_data["x9"])
a_x10 = np.array(copy_data["x10"])
a_x11 = np.array(copy_data["x11"])
a_x12 = np.array(copy_data["x12"])
a_x13 = np.array(copy_data["x13"])
a_type = np.array(copy_data["type"])

In [10]:
start = timer()
logistic_regression(2,a_x1,a_x2,a_x3,a_x4,a_x5,a_x6,a_x7,a_x8,a_x9,a_x10,a_x11,a_x12,a_x13,a_type)
print("without GPU:", timer() - start)

without GPU: 41.6967889


In [13]:
timer()

144.184023